In [1]:
# Import libraries
import os 
import ee
import time
import rasterio
import math
import pyproj
import libpysal
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
from rasterstats import zonal_stats
from pylandtemp import single_window
from shapely.geometry import Point
from shapely.ops import unary_union

<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).


In [2]:
directory = os.getcwd() #Location where files will be saved
c = "/"

In [5]:
# Download street network
G = ox.graph_from_place('Greater Manchester', network_type='all')

# Save to geopackage
ox.save_graph_geopackage(G, 'network.gpkg')

# Save to graphml
ox.io.save_graphml(G, 'network.graphml')

In [16]:
# Calculate distance variabels to POIs

bus_dist_list = []

city = 'Greater Manchester'
espg = 'EPSG:27700'
city_path = directory + c + city

df = gpd.read_file('smaller_section2.gpkg')
df = df.to_crs(espg)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

# Load in street network graph
G = ox.load_graphml('network.graphml')

# Project the graph to EPSG:27700
G = ox.project_graph(G, to_crs=espg)

# Load in bus stops
busstop = gpd.read_file('bus_stops.gpkg').to_crs(espg)

# Import points as gdf and loop through them
for index, point in gdf.iterrows():
    polygon_index = busstop.distance(point.geometry).sort_values().index[0]
    nearest_bus = busstop.loc[polygon_index].geometry.centroid

    # Extract coordinates of point
    x, y = point.geometry.centroid.coords.xy
    x = x[0]
    y = y[0]

    # Find closest node within threshold
    nearest_nodes, dist = ox.distance.nearest_nodes(G, x, y, return_dist=True)
    if isinstance(nearest_nodes, int):
        nearest_nodes = [nearest_nodes]
        dist = [dist]
    valid_nodes = [node for node, d in zip(nearest_nodes, dist) if d <= 1000]
    if valid_nodes:
        node_counter = valid_nodes[0]
    else:
        continue

    # Extract coordinates of point
    xx, yy = nearest_bus.xy
    xx = xx[0]
    yy = yy[0]

    # Find closest node within threshold
    nearest_nodes, dist = ox.distance.nearest_nodes(G, xx, yy, return_dist=True)
    if isinstance(nearest_nodes, int):
        nearest_nodes = [nearest_nodes]
        dist = [dist]
    valid_nodes = [node for node, d in zip(nearest_nodes, dist) if d <= 1000]
    if valid_nodes:
        node_bus = valid_nodes[0]
    else:
        continue

    # Calculate shortest path within threshold
    try:
        nx.shortest_path_length(G, node_counter, node_bus, weight='length')
    except:
        s_path = 0
    else:
        s_path = nx.shortest_path_length(G, node_counter, node_bus, weight='length')

    bus_dist_list.append(s_path)

print(city + ' is done')

df['dist_to_busstop'] = bus_dist_list
df

Greater Manchester is done


,dissolve_key,row3,col3,population,area_m,grid_lon,grid_lat,geometry,dist_to_busstop
0,244-582,244,582,4,5142,152029.569440,5.938764e+06,"POLYGON ((383755.864 398492.855, 383811.165 39...",97.296
1,244-583,244,583,5,5142,152084.801365,5.938760e+06,"POLYGON ((383811.165 398492.665, 383866.466 39...",77.717
2,244-584,244,584,5,5142,152140.033312,5.938756e+06,"POLYGON ((383866.466 398492.476, 383921.767 39...",152.121
3,244-585,244,585,6,5142,152195.265280,5.938752e+06,"POLYGON ((383921.767 398492.287, 383977.068 39...",461.462
4,244-586,244,586,10,5142,152250.497270,5.938748e+06,"POLYGON ((383977.068 398492.099, 384032.369 39...",205.684
...,...,...,...,...,...,...,...,...,...
669,269-602,269,602,59,5144,152963.967004,5.936368e+06,"POLYGON ((384854.467 396171.331, 384909.795 39...",0.000
670,269-603,269,603,10,5144,153019.226579,5.936364e+06,"POLYGON ((384909.795 396171.154, 384965.123 39...",75.919
671,269-604,269,604,5,5144,153074.486175,5.936360e+06,"POLYGON ((384965.123 396170.978, 385020.451 39...",165.514
672,269-605,269,605,6,5144,153129.745793,5.936356e+06,"POLYGON ((385020.451 396170.803, 385075.778 39...",69.629


In [7]:
df.to_file('popgriddistancetest.gpkg', driver='GPKG')